In [1]:
import requests
import json
import pprint
import pandas as pd
import csv
import re
from datetime import datetime
import time

- Get all shops

In [8]:
## nháp
page=1
# st=1
# all_shops_API = requests.get(f"https://www.foody.vn/ho-chi-minh/dia-diem?vt=row&st={st}&dt=4&page={page}", 
#                             headers={'X-Requested-With': 'XMLHttpRequest',
#                                      'Cookie': 'FOODY.AUTH=A3F33DD0BA0DC443960AC294DA1D898EE203A64E31B64523D3674F29D60EF3C3C900F1F3BF70DAC47FD0ED268F012BD8FD06CCA143E3FCC2F79AA22CFF031D91A06F5D97B96B09529CF8E5A06E58BCF34312AFDA1F37E6B39CC56A6BA525D72DEDDF1EFE136334F777A1AD191BC3B393CA5D7B3008AA54A9565C21793E196AB15D19E8867327553F696DDCA2938C22208B84620BAA2F757C9D4B8E2FF3CE8A28E6EBC891AAE2F227C02CF6834427BD815163F0DAB98F2E47CECEA44EE3CB88F9A94D33123EEA15B660CF6DE09385DF9EAFD8741C8B4936BF4B5C40B003150E622D303BAC626EBF7FD60BCB19647A46FC',
#                                     })
# data = all_shops_API.text 
# dataJson = json.loads(data)

# print(len(dataJson['searchItems']))
# dataJson

# a = get_single_page(page, 100, 4)
# b = get_single_page(page, 10, 4)
# bruh = []
# bruh.extend(a)
# bruh.extend(b)
# bruh
# df = pd.DataFrame.from_dict(bruh) 
# df.drop_duplicates()
# len(bruh)

24

In [37]:
def get_single_page(page, st, dist):
    '''extract 1 page'''
    url = f'https://www.foody.vn/ho-chi-minh/dia-diem?vt=row&st={st}&dt={dist}&page={page}'
    all_shops_API = requests.get(url, headers={'X-Requested-With': 'XMLHttpRequest',
                                               'Cookie': 'FOODY.AUTH=A3F33DD0BA0DC443960AC294DA1D898EE203A64E31B64523D3674F29D60EF3C3C900F1F3BF70DAC47FD0ED268F012BD8FD06CCA143E3FCC2F79AA22CFF031D91A06F5D97B96B09529CF8E5A06E58BCF34312AFDA1F37E6B39CC56A6BA525D72DEDDF1EFE136334F777A1AD191BC3B393CA5D7B3008AA54A9565C21793E196AB15D19E8867327553F696DDCA2938C22208B84620BAA2F757C9D4B8E2FF3CE8A28E6EBC891AAE2F227C02CF6834427BD815163F0DAB98F2E47CECEA44EE3CB88F9A94D33123EEA15B660CF6DE09385DF9EAFD8741C8B4936BF4B5C40B003150E622D303BAC626EBF7FD60BCB19647A46FC'})
    data = all_shops_API.text
    json_data = json.loads(data)
    if not json_data['searchItems']:
        return []
    shop_attribs = []
    
    for shop in json_data['searchItems']: 
        shop_attribs.append({'ReviewUrl' : shop['ReviewUrl'],
                             'name': shop['Name'],
                             'Address': shop['Address'],
                             'districtid': shop['DistrictId'],
                             'RestaurantId': shop['Id'],
                             'RestaurantStatus': shop['Status'],
                             'avgrating' : shop['AvgRatingOriginal'],
                             'TotalPictures' : shop['TotalPictures'],
                             'TotalSaves' : shop['TotalSaves'],
                             'TotalReviews' : shop['TotalReview'],
                             'TotalFavourites' : shop['TotalFavourite'],
                             'TotalViews' : shop['TotalView'],
                             'TotalCheckIns' : shop['TotalCheckins'],
                             'Latitude'  : shop['Latitude'],
                             'Longitude'  : shop['Longitude']}) # bsung 
    
    return shop_attribs # [{},{},..]

def get_single_district(dist=1):
    '''get all shops in a single district'''
    page = 1
    all_shop = []
    empty = False

    for page in range(1,80):
        if empty:
            break
        print(f'page {page}')
        st = 1

        for st in range(1,50):
            print(f'  st = {st}')
            one_page = get_single_page(page, st, dist) 
            time.sleep(1)
            if not one_page: # check empty
                empty = True
                break
            all_shop.extend(one_page)
    return all_shop # [{},{},..]                       

# def get_single_city(city_id='217'):
#     all_dist = '1,4,5,6,7,8,9,10,11,12,13,14,15,16,17,19,2,18,693,696,699,695,694,698'.split(',')

#     for dist in all_dist:
# lmao = get_single_district()
# 9612
# lọc trùng: (674, 15)
# 8532
# lọc trùng: (675, 15)
# 7452
# lọc trùng: (671, 15)

In [38]:
shops_in_a_district = get_single_district(dist=4)
print(len(shops_in_a_district))

df = pd.DataFrame.from_dict(shops_in_a_district) 
print(f'lọc trùng:', df.drop_duplicates().shape)
df.drop_duplicates().to_csv(r'allshop.csv', index = False, header=True, encoding='utf-8')

page 1
  st = 1
  st = 2
  st = 3
  st = 4
  st = 5
  st = 6
  st = 7
  st = 8
  st = 9
  st = 10
  st = 11
  st = 12
  st = 13
  st = 14
  st = 15
  st = 16
  st = 17
  st = 18
  st = 19
  st = 20
  st = 21
  st = 22
  st = 23
  st = 24
  st = 25
  st = 26
  st = 27
  st = 28
  st = 29
  st = 30
  st = 31
  st = 32
  st = 33
  st = 34
  st = 35
  st = 36
  st = 37
  st = 38
  st = 39
  st = 40
  st = 41
  st = 42
  st = 43
  st = 44
  st = 45
  st = 46
  st = 47
  st = 48
  st = 49
page 2
  st = 1
  st = 2
  st = 3
  st = 4
  st = 5
  st = 6
  st = 7
  st = 8
  st = 9
  st = 10
  st = 11
  st = 12
  st = 13
  st = 14
  st = 15
  st = 16
  st = 17
  st = 18
  st = 19
  st = 20
  st = 21
  st = 22
  st = 23
  st = 24
  st = 25
  st = 26
  st = 27
  st = 28
  st = 29
  st = 30
  st = 31
  st = 32
  st = 33
  st = 34
  st = 35
  st = 36
  st = 37
  st = 38
  st = 39
  st = 40
  st = 41
  st = 42
  st = 43
  st = 44
  st = 45
  st = 46
  st = 47
  st = 48
  st = 49
page 3
  st = 1
  st = 2

  st = 39
  st = 40
  st = 41
  st = 42
  st = 43
  st = 44
  st = 45
  st = 46
  st = 47
  st = 48
  st = 49
page 18
  st = 1
  st = 2
  st = 3
  st = 4
  st = 5
  st = 6
  st = 7
  st = 8
  st = 9
  st = 10
  st = 11
  st = 12
  st = 13
  st = 14
  st = 15
  st = 16
  st = 17
  st = 18
  st = 19
  st = 20
  st = 21
  st = 22
  st = 23
  st = 24
  st = 25
  st = 26
  st = 27
  st = 28
  st = 29
  st = 30
  st = 31
  st = 32
  st = 33
  st = 34
  st = 35
  st = 36
  st = 37
  st = 38
  st = 39
  st = 40
  st = 41
  st = 42
  st = 43
  st = 44
  st = 45
  st = 46
  st = 47
  st = 48
  st = 49
page 19
  st = 1
  st = 2
  st = 3
  st = 4
  st = 5
  st = 6
  st = 7
  st = 8
  st = 9
  st = 10
  st = 11
  st = 12
  st = 13
  st = 14
  st = 15
  st = 16
  st = 17
  st = 18
  st = 19
  st = 20
  st = 21
  st = 22
  st = 23
  st = 24
  st = 25
  st = 26
  st = 27
  st = 28
  st = 29
  st = 30
  st = 31
  st = 32
  st = 33
  st = 34
  st = 35
  st = 36
  st = 37
  st = 38
  st = 39
  st = 40
  

  st = 27
  st = 28
  st = 29
  st = 30
  st = 31
  st = 32
  st = 33
  st = 34
  st = 35
  st = 36
  st = 37
  st = 38
  st = 39
  st = 40
  st = 41
  st = 42
  st = 43
  st = 44
  st = 45
  st = 46
  st = 47
  st = 48
  st = 49
page 35
  st = 1
  st = 2
  st = 3
  st = 4
  st = 5
  st = 6
  st = 7
  st = 8
  st = 9
  st = 10
  st = 11
  st = 12
  st = 13
  st = 14
  st = 15
  st = 16
  st = 17
  st = 18
  st = 19
  st = 20
  st = 21
  st = 22
  st = 23
  st = 24
  st = 25
  st = 26
  st = 27
  st = 28
  st = 29
  st = 30
  st = 31
  st = 32
  st = 33
  st = 34
  st = 35
  st = 36
  st = 37
  st = 38
  st = 39
  st = 40
  st = 41
  st = 42
  st = 43
  st = 44
  st = 45
  st = 46
  st = 47
  st = 48
  st = 49
page 36
  st = 1
  st = 2
  st = 3
  st = 4
  st = 5
  st = 6
  st = 7
  st = 8
  st = 9
  st = 10
  st = 11
  st = 12
  st = 13
  st = 14
  st = 15
  st = 16
  st = 17
  st = 18
  st = 19
  st = 20
  st = 21
  st = 22
  st = 23
  st = 24
  st = 25
  st = 26
  st = 27
  st = 28
  

  st = 15
  st = 16
  st = 17
  st = 18
  st = 19
  st = 20
  st = 21
  st = 22
  st = 23
  st = 24
  st = 25
  st = 26
  st = 27
  st = 28
  st = 29
  st = 30
  st = 31
  st = 32
  st = 33
  st = 34
  st = 35
  st = 36
  st = 37
  st = 38
  st = 39
  st = 40
  st = 41
  st = 42
  st = 43
  st = 44
  st = 45
  st = 46
  st = 47
  st = 48
  st = 49
page 52
  st = 1
  st = 2
  st = 3
  st = 4
  st = 5
  st = 6
  st = 7
  st = 8
  st = 9
  st = 10
  st = 11
  st = 12
  st = 13
  st = 14
  st = 15
  st = 16
  st = 17
  st = 18
  st = 19
  st = 20
  st = 21
  st = 22
  st = 23
  st = 24
  st = 25
  st = 26
  st = 27
  st = 28
  st = 29
  st = 30
  st = 31
  st = 32
  st = 33
  st = 34
  st = 35
  st = 36
  st = 37
  st = 38
  st = 39
  st = 40
  st = 41
  st = 42
  st = 43
  st = 44
  st = 45
  st = 46
  st = 47
  st = 48
  st = 49
page 53
  st = 1
  st = 2
  st = 3
  st = 4
  st = 5
  st = 6
  st = 7
  st = 8
  st = 9
  st = 10
  st = 11
  st = 12
  st = 13
  st = 14
  st = 15
  st = 16
  

  st = 3
  st = 4
  st = 5
  st = 6
  st = 7
  st = 8
  st = 9
  st = 10
  st = 11
  st = 12
  st = 13
  st = 14
  st = 15
  st = 16
  st = 17
  st = 18
  st = 19
  st = 20
  st = 21
  st = 22
  st = 23
  st = 24
  st = 25
  st = 26
  st = 27
  st = 28
  st = 29
  st = 30
  st = 31
  st = 32
  st = 33
  st = 34
  st = 35
  st = 36
  st = 37
  st = 38
  st = 39
  st = 40
  st = 41
  st = 42
  st = 43
  st = 44
  st = 45
  st = 46
  st = 47
  st = 48
  st = 49
page 69
  st = 1
  st = 2
  st = 3
  st = 4
  st = 5
  st = 6
  st = 7
  st = 8
  st = 9
  st = 10
  st = 11
  st = 12
  st = 13
  st = 14
  st = 15
  st = 16
  st = 17
  st = 18
  st = 19
  st = 20
  st = 21
  st = 22
  st = 23
  st = 24
  st = 25
  st = 26
  st = 27
  st = 28
  st = 29
  st = 30
  st = 31
  st = 32
  st = 33
  st = 34
  st = 35
  st = 36
  st = 37
  st = 38
  st = 39
  st = 40
  st = 41
  st = 42
  st = 43
  st = 44
  st = 45
  st = 46
  st = 47
  st = 48
  st = 49
page 70
  st = 1
  st = 2
  st = 3
  st = 4
  st

---

In [89]:
def extractshop(page, page, st, dist):
    '''extract 1 page'''
    url = f'https://www.foody.vn/ho-chi-minh/dia-diem?vt=row&st={st}&dt={dist}&page={page}'
    all_shops_API = requests.get(url, headers={'X-Requested-With': 'XMLHttpRequest',
                                               'Cookie': 'FOODY.AUTH=A3F33DD0BA0DC443960AC294DA1D898EE203A64E31B64523D3674F29D60EF3C3C900F1F3BF70DAC47FD0ED268F012BD8FD06CCA143E3FCC2F79AA22CFF031D91A06F5D97B96B09529CF8E5A06E58BCF34312AFDA1F37E6B39CC56A6BA525D72DEDDF1EFE136334F777A1AD191BC3B393CA5D7B3008AA54A9565C21793E196AB15D19E8867327553F696DDCA2938C22208B84620BAA2F757C9D4B8E2FF3CE8A28E6EBC891AAE2F227C02CF6834427BD815163F0DAB98F2E47CECEA44EE3CB88F9A94D33123EEA15B660CF6DE09385DF9EAFD8741C8B4936BF4B5C40B003150E622D303BAC626EBF7FD60BCB19647A46FC'})
    data = all_shops_API.text
    json_data = json.loads(data)
    shop_attribs = []
    
    for shop in json_data['searchItems']: 
        shop_review.append({'ReviewUrl' : shop['ReviewUrl'],
                            'name': shop['Name'],
                            'Address': shop['Address'],
                            'RestaurantId': shop['RestaurantId'],
                            'RestaurantStatus': shop['RestaurantStatus'],
                            'avgrating' : shop['AvgRating'],
                            'TotalPictures' : shop['TotalPictures'],
                            'TotalSaves' : shop['TotalSaves'],
                            'TotalReviews' : shop['TotalReviews'],
                            'TotalFavourites' : shop['TotalFavourites'],
                            'TotalViews' : shop['TotalViews'],
                            'TotalCheckIns' : shop['TotalCheckIns'],
                            'Latitude'  : shop['Latitude'],
                            'Longitude'  : shop['Longitude']})

        # converted_date = shop['ResCreatedOn']
        # converted_date = converted_date[7:-2]
        # converted_date = int(converted_date)
        # converted_date = datetime.fromtimestamp(converted_date // 1000000000)
        # converted_date = converted_date.strftime('%d/%m/%Y')
        # shop_review.append({'CityId': CityId, 
        #                     'name': shop['Name'],
        #                     'addr': shop['Address'],
        #                     'RestaurantId': shop['RestaurantId'],
        #                     'RestaurantStatus': shop['RestaurantStatus'],
        #                     'avgrating' : shop['AvgRating'],
        #                     'TotalPictures' : shop['TotalPictures'],
        #                     'TotalSaves' : shop['TotalSaves'],
        #                     'TotalReviews' : shop['TotalReviews'],
        #                     'TotalFavourites' : shop['TotalFavourites'],
        #                     'TotalViews' : shop['TotalViews'],
        #                     'TotalCheckIns' : shop['TotalCheckIns'],
        #                     'CreatedOn'  : converted_date,
        #                     'Latitude'  : shop['Latitude'],
        #                     'Longitude'  : shop['Longitude'],
        #                     'ResUrlReviews' : shop['ResUrlReviews']})

    return shop_review

def extractshop_all():
    page = 1
    all_shop = []
    count = 550

    while page < 100:
        print(f'page {page}')
        one_page = extractshop(page, count)
        all_shop.extend(one_page)

        # if len(one_page) < count:
        #     break
        # else:
        page += 1
        time.sleep(1)
    return all_shop

In [ ]:
lmao = extractshop_all()
print(len(lmao))

df = pd.DataFrame.from_dict(lmao) 
print(f'lọc trùng:', df.drop_duplicates().shape)

In [70]:
df = pd.DataFrame.from_dict(lmao) 
df.to_csv (r'allshop.csv', index = False, header=True, encoding='utf-8')

In [40]:
## 
df.drop_duplicates().shape

(2000, 16)

- Get menu of ONE shop: fải loop qua từng request_id trong cell code trên

In [ ]:
# VD: có thể thay request_id = 1031704 để thử
request_id = 1031704
delivery_dish_API = requests.get(f"https://gappapi.deliverynow.vn/api/dish/get_delivery_dishes?request_id={request_id}&id_type=1", 
                                    headers={'x-foody-api-version': '1',
                                            'x-foody-app-type': '1004',
                                            'x-foody-client-language': 'vi',
                                            'x-foody-client-type': '1',
                                            'x-foody-client-version': '1',
                                            'X-Foody-Client-Id': 'cookies.__ondemand_sessionid'
                                        }
                                )
data = delivery_dish_API.text
dataJson = json.loads(data)
print(dataJson)

- get detail of ONE shop

In [ ]:
delivery_dish_API = requests.get("https://gappapi.deliverynow.vn/api/delivery/get_detail?request_id={request_id}&id_type=1", 
                                    headers={'x-foody-api-version': '1',
                                            'x-foody-app-type': '1004',
                                            'x-foody-client-language': 'vi',
                                            'x-foody-client-type': '1',
                                            'x-foody-client-version': '1',
                                            'X-Foody-Client-Id': 'cookies.__ondemand_sessionid'
                                        }
                                )
data = delivery_dish_API.text
dataJson = json.loads(data)
print(dataJson)